In [50]:
import pandas as pd
import numpy as np 

X_train = pd.read_parquet("preprocessed/X_train.parquet")
y_train = pd.read_parquet("preprocessed/y_train.parquet")
X_test = pd.read_parquet("preprocessed/X_test.parquet")
y_test = pd.read_parquet("preprocessed/y_test.parquet")

In [51]:
X_train = X_train["text"]
X_test = X_test["text"]

In [52]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
le.fit(y_train.values)
le.classes_

c:\Users\ciril\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['БАР', 'ОКР', 'ПРЛ', 'депрессия', 'тревожное р-во', 'шизофрения'],
      dtype=object)

In [53]:
y_train = le.transform(y_train.values.ravel())
y_test = le.transform(y_test.values.ravel())

In [54]:
y_test

array([1, 3, 1, ..., 2, 3, 0])

In [55]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

In [56]:
# Загрузка токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Токенизация данных
train_encodings = tokenizer(list(X_train), return_tensors="pt", truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), return_tensors="pt", truncation=True, padding=True, max_length=512)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
np.array([1])

array([1])

In [64]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #print(item)
        # print(self.labels[idx])
        item['labels'] = torch.LongTensor(np.array([self.labels[idx]])).unsqueeze(0)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)

In [65]:
# Настройка тренера
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Обучение модели
trainer.train()

c:\Users\ciril\anaconda3\lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/6051 [00:00<?, ?it/s]

C:\Users\ciril\AppData\Local\Temp/ipykernel_8224/2715799407.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 1.7938, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 1.7723, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 1.763, 'learning_rate': 3e-06, 'epoch': 0.01}
{'loss': 1.7231, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 1.7049, 'learning_rate': 5e-06, 'epoch': 0.02}
{'loss': 1.6899, 'learning_rate': 6e-06, 'epoch': 0.03}
{'loss': 1.6323, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.03}
{'loss': 1.6435, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.04}
{'loss': 1.7701, 'learning_rate': 9e-06, 'epoch': 0.04}


KeyboardInterrupt: 